<a href="https://colab.research.google.com/github/gmortuza/machine-learning-scratch/blob/master/machine_learning/bayesian/gaussian_discriminative_analysis/Gaussian%20Discriminative%20analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import cm
from mpl_toolkits.mplot3d import Axes3D
from sklearn.datasets import load_iris
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from scipy.stats import multivariate_normal
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets

# Gaussian Discriminative Analysis
### Generative Learning Algorithm vs Discriminative Learning Algorithm
Algorithms that try to learn mappings from input space $ \mathcal{x} $ to the output labels $ \mathcal{y} $ ( such as logistic regression, linear regression, etc.) are called **Discriminative Learning Algorithm(DLA).** In other words, the discriminative learning algorithm tries to learn $ y $ given $ x $. Mathematically $ P(y | x) $. On the other hand, algorithm that tries to learn $ x $ given $ y $ $ (i. e. p(x | y)) $ is called **Generative Learning Algorithm (GLA)**. Naive Bayes, Gaussian discriminant analysis are the example of GLA. While DLA tries to find a decision boundary based on the input data, GLA tries to fit a gaussian in each output label.
![DLA vs GLA](https://github.com/gmortuza/machine-learning-scratch/blob/master/machine_learning/bayesian/gaussian_discriminative_analysis/img/GLA_DLA.png?raw=1)

### Multivariate Gaussian Distribution
Gaussian Discriminant Analysis model assumes that $ p(x | y) $ is distributed according to a multivariate normal distribution, which is parameterized by a **mean vector** $ \mu \in \mathbb{R}^n $ and a **covariance matrix** $ \Sigma \in \mathbb{R}^{nxn} $. This can also be written as $ \mathcal{N}(\mu, \Sigma) $ and it's density function can be given by:
$$ 
p(x;\mu,\Sigma) = \frac{1}{(2\pi)^{n/2}|\Sigma|^{1/2}}exp\Bigg(-\frac{1}{2}\big(x - \mu\big)^T\Sigma^{-1}\big(x - \mu\big)\Bigg)
$$
The mean vector and covariance matrix will determine the shape of the probability density function. In the following section you can play with these parameter.

In [ ]:
@interact(mu1=(-3,3,0.1),  mu2=(-3,3.0,0.1), diagonal_1=(0,3.0,0.1), diagonal_2=(0,3.0,0.1), non_diagonal=(-3,3.0,0.1))
def visualize_multivariate_gaussian(mu1=0.0, mu2=0.0, diagonal_1=1, diagonal_2=1, non_diagonal=0):
    # This code snippet is taken from here [https://scipython.com/blog/visualizing-the-bivariate-gaussian-distribution/]
    N = 300
    X = np.linspace(-3, 3, N)
    Y = np.linspace(-3, 4, N)
    X, Y = np.meshgrid(X, Y)

    # Mean vector and covariance matrix
    mu = np.array([mu1, mu2])
    Sigma = np.array([[ diagonal_1 , non_diagonal], [non_diagonal,  diagonal_2]])
    print("𝜇 = ", mu)
    print("Σ = ", Sigma)
    # Pack X and Y into a single 3-dimensional array
    pos = np.empty(X.shape + (2,))
    pos[:, :, 0] = X
    pos[:, :, 1] = Y

    def multivariate_gaussian(pos, mu, Sigma):
        """Return the multivariate Gaussian distribution on array pos.

        pos is an array constructed by packing the meshed arrays of variables
        x_1, x_2, x_3, ..., x_k into its _last_ dimension.

        """

        n = mu.shape[0]
        Sigma_det = np.linalg.det(Sigma)
        Sigma_inv = np.linalg.inv(Sigma)
        N = np.sqrt((2*np.pi)**n * Sigma_det)
        # This einsum call calculates (x-mu)T.Sigma-1.(x-mu) in a vectorized
        # way across all the input variables.
        fac = np.einsum('...k,kl,...l->...', pos-mu, Sigma_inv, pos-mu)

        return np.exp(-fac / 2) / N

    # The distribution on the variables X, Y packed into pos.
    Z = multivariate_gaussian(pos, mu, Sigma)

    # Create a surface plot and projected filled contour plot under it.
    fig = plt.figure(figsize=(15,10))
    ax = fig.gca(projection='3d')
    ax.plot_surface(X, Y, Z, rstride=3, cstride=3, linewidth=1, antialiased=True,
                    cmap=cm.viridis)

    cset = ax.contourf(X, Y, Z, zdir='z', offset=-0.15, cmap=cm.viridis)

    # Adjust the limits, ticks and view angle
    ax.set_zlim(-0.15,0.2)
    ax.set_zticks(np.linspace(0,0.2,5))
    ax.view_init(27, -21)

    plt.show()
    

interactive(children=(FloatSlider(value=0.0, description='mu1', max=3.0, min=-3.0), FloatSlider(value=0.0, des…

![Multivariate gaussian](https://github.com/gmortuza/machine-learning-scratch/blob/master/machine_learning/bayesian/gaussian_discriminative_analysis/img/multivariate_gaussian.png?raw=1)

### Gaussian Discriminant Analysis(GDA) model
GDA is perfect for the case where the problem is a classificaiton problem and input variable is continious and fall into a gaussian distribution. Now lets make a flower classifier model using the iris dataset. We will apply GDA model which will model $ p(x|y) $ using a multivariate normal distribution. Iris dataset have 3 labels/class. Those are setosa, versicolor and virginica. For mathemetical modelling we will denote setosa as class 0, versicolor as class 1, virginica as class 2.
During the training process at first we will calculate the **class probability** for each class. Class probability indicates how often data is present in the training set. For example, if we have 100 training example and class 'versicolor' appears 13 times then the class probability for class 'versicolor' will be $ \phi_1 = \frac{13}{100} = 0.13 $. So generally,
$$
\begin{align}
\phi_k &= \frac{\text{number of occurance of class k}}{\text{total number of training example}} \\
        &= \frac{1}{m}\sum_{i=1}^m1\{y^{(i)} = k \}
\end{align}
$$
Here *m* is the number of training example, *k* is a specific class and $ \phi_k $ represents the class probability of class *k*.

Now we will fit a gaussian in each of the classes. For example, if we fit a gaussian for all the training data for class 0 we get it's density function as:
$$
p(x | y=0) \sim \mathcal{N}(\mu_0, \Sigma_0)
$$
Here $ \mu_0 $ is the mean and $ \Sigma_0) $ is the covariance matrix for all the training set of class 0. More generally for class *k* we get the density function:
$$
p(x | y=k) \sim \mathcal{N}(\mu_k, \Sigma_k) = \frac{1}{(2\pi)^{n/2}|\Sigma_k|^{1/2}}exp\Bigg(-\frac{1}{2}\big(x - \mu_k\big)^T\Sigma_k^{-1}\big(x - \mu_k\big)\Bigg)
$$

This density function is parameterized by mean($ \mu $) and covariance matrix($ \Sigma $). So, we need to find mean and covariance matrix for all classes. We can find mean $ \mu $ and covariance matrix, $ \Sigma $ for class *k* using the following equation.
$$ 
\mu_k = \frac{\sum_{i=1}^m 1 \{y^{(i)} = k\} x^{(i)}}{\sum_{i=1}^m 1 \{y^{(i)} = k\}} \\
\Sigma = \frac{1}{m}\sum_{i=1}^m \big(x^{(i)} - \mu_{y^{(i)}}\big) \big(x^{(i)} - \mu_{y^{(i)}}\big)^T
$$

Before going into the prediction let's implement the theory that we discussed so far.

In [ ]:
def fit(x_train, y_train):
    m = y_train.shape[0] # Number of training example
    #Reshapeing the training set
    x_train = x_train.reshape(m, -1)
    input_feature = x_train.shape[1] # Number of input feature. In our case it's 4
    class_label = len(np.unique(y_train.reshape(-1))) # Number of class. In our case its 3.
    
    # Start everything with zero first.
    # Mean for each class. Each row contains an individual class. And each of the class input is 4 dimenstional
    mu = np.zeros((class_label, input_feature))
    # Each row will conatain the covariance matrix of each class.
    # The covariance matrix is a square symettric matrix.
    # It indicates how each of the input feature varies with each other.
    sigma = np.zeros((class_label, input_feature, input_feature))
    # Prior probability of each class.
    # Its the measure of knowing the likelihood of any class before seeing the input data.
    phi = np.zeros(class_label)

    for label in range(class_label):
        # Seperate all the training data for a single class
        indices = (y_train == label)
        
        phi[label] = float(np.sum(indices)) / m
        mu[label] = np.mean(x_train[indices, :], axis=0)
        # Instead of writting the equation we used numpy covariance function. 
        sigma[label] = np.cov(x_train[indices, :], rowvar=0)
    
    return phi, mu, sigma

### Making prediction:

Now let's assume we have some data *'x_test'* that we want to predict. First we will match our data with each of the class. We will use the gaussian distribution($ \mu, \Sigma $) of the classes that we calculated during the training process. This will give us the probability of *'x_test'* being in a specific class. For example, the following equation will give us the probability of *x_text* being the class 0.

$$
p(y=0 | x\_test) = \frac{1}{(2\pi)^{n/2}|\Sigma_0|^{1/2}}exp\Bigg(-\frac{1}{2}\big(x\_test - \mu_0\big)^T\Sigma_0^{-1}\big(x\_test - \mu_0\big)\Bigg)
$$

We will calculate $ p(y=k|x\_test) $ for each class k, where $ k \in \{0, 1, 2\} $. During the training we also calculated the class probability($ \phi $) of each of the class. We will multiply class probability with the gaussian probability to get the overall probability of a class. 

$$ p(y=k) \rightarrow p(y=k| x\_test) \times \phi_k $$

For whichever classes the probability value is higest we will consider that as the *x_test's* class. So we will maximize $ p(y=k) $. Probability can be a small number. So its better to maximum the log-likelihood of the data. 
$$ log\big(p(y=k)\big) = \big(p(y=k| x\_test)\big) + log\big(\phi_k\big) $$.
Lets say, for different classes we got the following probability:


<div style="font-size:20px;margin-top:15px;">
    <ul>
        <li>setosa(0) $ \rightarrow log\big(\phi_0\big) \times  log\big(p(y=0 | x\_test\big) = -1.05 + (-169.74) = -170.80 $</li>
<li>versicolor(1) $ \rightarrow log\big(\phi_1\big) \times  log\big(p(y=1 | x\_test\big) = -1.10 + (-1.05) = -2.16 $</li>
<li>virginica(2) $ \rightarrow log\big(\phi_2\big) \times  log\big(p(y=2 | x\_test\big) = -1.13 + (-10.15) = -11.28 $</li>
        </ul>
</div>

Here, class versicolor(1) got the highest value. So the prediction of *x_test* will be versicolor. Now lets implement our prediction function.

In [ ]:
def predict(x_tests, phi, mu, sigma):
    # flatten the training data
    x_tests = x_tests.reshape(x_tests.shape[0], -1)
    class_label = mu.shape[0] # Number of label we have in our case it's k = 3
    scores = np.zeros((x_tests.shape[0], class_label))  # Initially we set the each class probability to zero.
    for label in range(class_label): # We will calculate the probability for each of the class.
        # normal_distribution_prob.logpdf Will give us the log value of the PDF that we just mentioned above.
        normal_distribution_prob = multivariate_normal(mean=mu[label], cov=sigma[label])
        # x_test can have multiple test data we will calculate the probability of each of the test data
        for i, x_test in enumerate(x_tests):
            scores[i, label] = np.log(phi[label]) + normal_distribution_prob.logpdf(x_test)
    predictions = np.argmax(scores, axis=1)
    return predictions

### Test our model

In [ ]:
data = load_iris()
x_train, x_test, y_train, y_test = train_test_split(data.data, data.target)
phi, mu, sigma = fit(x_train, y_train)
y_predict = predict(x_test, phi, mu, sigma)
score = f1_score(y_test, y_predict, average="weighted")
print("f1 score of our model: ", score)

# Compare this model with scikitlearn LinearDiscriminatorAnalysis
lda = LinearDiscriminantAnalysis()
lda.fit(x_train, y_train)
y_predict_sk = lda.predict(x_test)
print("f1 score of scikit-learn model is: ", f1_score(y_test, y_predict_sk, average="weighted"))

f1 score of our model:  1.0
f1 score of scikit-learn model is:  1.0


### Conclusion
One of biggest advantages of GLA model is it doesn't have any hyperparameter. This model works really well if the input dataset follows the gaussian distribution. If all the class share the same covariance matrix then the model is called Linear Discriminant Analysis(LDA) and if each class have different covariance matrix then the model called Quadratic Discriminant Analysis(QDA). 
Both the Logistic regression and GDA are classification algorithm and they share an interesting relationship. If we view the quantity of $ p(y=1 |x; \phi_k, \mu_k, \Sigma_k) $ as a function of x we will get the logistic/sigmoid function. So, when would we prefer one model over another? 

**GDA**
- makes stronger modeling assumptions 
- requies less data
- works great if the input data follows gaussian distribution.
- Have no hyperparameter

**Logistic regression**
- makes weaker assumption about the data.
- requires larger dataset
- works better than GDA if the input data are non-gaussian thats why this algorithm used often than GDA
- Have learning rate as a hyperparameter

#### GDA as dimensionality reduction

Like PCA(Principal Component Analysis) this model can be used to reduce the dimensionality of the data as well. PCA finds the axes with maximum variance for the whole data set where LDA tries to find the axes for best class seperability. In practice, often a LDA is done followed by a PCA for dimensionality reduction.
![LDA vs PCA](https://github.com/gmortuza/machine-learning-scratch/blob/master/machine_learning/bayesian/gaussian_discriminative_analysis/img/LDA_PCA.png?raw=1)
taken from: https://sebastianraschka.com/Articles/2014_python_lda.html
